# मेटा फैमिली मॉडल्स के साथ निर्माण

## परिचय

इस पाठ में आप सीखेंगे:

- मेटा फैमिली के दो मुख्य मॉडल्स - Llama 3.1 और Llama 3.2 का अन्वेषण
- हर मॉडल के उपयोग के मामले और परिदृश्य समझना
- हर मॉडल की खासियतें दिखाने के लिए कोड का उदाहरण

## मेटा फैमिली के मॉडल्स

इस पाठ में हम मेटा फैमिली या "Llama Herd" के 2 मॉडल्स - Llama 3.1 और Llama 3.2 को जानेंगे

ये मॉडल्स अलग-अलग वेरिएंट्स में आते हैं और Github Model मार्केटप्लेस पर उपलब्ध हैं। Github Models का उपयोग करके [AI मॉडल्स के साथ प्रोटोटाइपिंग](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) के बारे में अधिक जानकारी यहाँ है।

मॉडल वेरिएंट्स:
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*नोट: Llama 3 भी Github Models पर उपलब्ध है, लेकिन इस पाठ में इसे शामिल नहीं किया गया है*


## Llama 3.1

405 बिलियन पैरामीटर्स के साथ, Llama 3.1 ओपन सोर्स LLM श्रेणी में आता है।

यह मॉडल पहले के Llama 3 वर्शन का अपग्रेड है और इसमें ये खासियतें हैं:

- बड़ा कॉन्टेक्स्ट विंडो - 128k टोकन बनाम 8k टोकन
- अधिकतम आउटपुट टोकन की सीमा बढ़ी - 4096 बनाम 2048
- बेहतर मल्टीलिंगुअल सपोर्ट - ट्रेनिंग टोकन की संख्या बढ़ने के कारण

इन खूबियों के कारण Llama 3.1 जेनएआई एप्लिकेशन बनाते समय और जटिल उपयोग मामलों को संभाल सकता है, जैसे कि:
- नेटिव फंक्शन कॉलिंग - LLM वर्कफ़्लो के बाहर एक्सटर्नल टूल्स और फंक्शन्स को कॉल करने की क्षमता
- बेहतर RAG परफॉर्मेंस - बड़े कॉन्टेक्स्ट विंडो के कारण
- सिंथेटिक डेटा जेनरेशन - फाइन-ट्यूनिंग जैसे कार्यों के लिए प्रभावी डेटा बनाने की क्षमता


### नेटिव फंक्शन कॉलिंग

Llama 3.1 को फंक्शन या टूल कॉल करने में और अधिक प्रभावी बनाने के लिए फाइन-ट्यून किया गया है। इसमें दो बिल्ट-इन टूल्स भी हैं जिन्हें मॉडल यूज़र के प्रॉम्प्ट के आधार पर इस्तेमाल करने की जरूरत पहचान सकता है। ये टूल्स हैं:

- **Brave Search** - वेब सर्च करके मौसम जैसी ताज़ा जानकारी प्राप्त करने के लिए इस्तेमाल किया जा सकता है
- **Wolfram Alpha** - जटिल गणितीय गणनाओं के लिए इस्तेमाल किया जा सकता है, जिससे आपको खुद के फंक्शन लिखने की जरूरत नहीं पड़ती।

आप अपने खुद के कस्टम टूल्स भी बना सकते हैं जिन्हें LLM कॉल कर सकता है।

नीचे दिए गए कोड उदाहरण में:

- हम सिस्टम प्रॉम्प्ट में उपलब्ध टूल्स (brave_search, wolfram_alpha) को परिभाषित करते हैं।
- एक यूज़र प्रॉम्प्ट भेजते हैं जिसमें किसी खास शहर के मौसम के बारे में पूछा जाता है।
- LLM Brave Search टूल को कॉल करने के लिए जवाब देगा, जो इस तरह दिखेगा `<|python_tag|>brave_search.call(query="Stockholm weather")`

*नोट: यह उदाहरण केवल टूल कॉल करता है, अगर आप परिणाम प्राप्त करना चाहते हैं, तो आपको Brave API पेज पर एक फ्री अकाउंट बनाना होगा और फंक्शन को खुद परिभाषित करना होगा*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

हालाँकि Llama 3.1 एक LLM है, इसकी एक सीमा मल्टीमोडैलिटी है। यानी, अलग-अलग प्रकार के इनपुट जैसे कि इमेज को प्रॉम्प्ट के रूप में इस्तेमाल करना और उसके आधार पर जवाब देना। यह क्षमता Llama 3.2 की मुख्य विशेषताओं में से एक है। इन विशेषताओं में शामिल हैं:

- मल्टीमोडैलिटी - इसमें टेक्स्ट और इमेज दोनों प्रॉम्प्ट को समझने की क्षमता है
- छोटे से मध्यम आकार के वेरिएंट (11B और 90B) - इससे फ्लेक्सिबल डिप्लॉयमेंट के विकल्प मिलते हैं,
- केवल टेक्स्ट वेरिएंट (1B और 3B) - इससे मॉडल को एज / मोबाइल डिवाइस पर डिप्लॉय किया जा सकता है और कम लेटेंसी मिलती है

मल्टीमोडल सपोर्ट ओपन सोर्स मॉडल्स की दुनिया में एक बड़ा कदम है। नीचे दिए गए कोड उदाहरण में एक इमेज और टेक्स्ट प्रॉम्प्ट दोनों का उपयोग किया गया है ताकि Llama 3.2 90B से इमेज का विश्लेषण प्राप्त किया जा सके।

### Llama 3.2 के साथ मल्टीमोडल सपोर्ट


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## सीखना यहीं नहीं रुकता, अपनी यात्रा जारी रखें

इस पाठ को पूरा करने के बाद, हमारे [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) को देखें और अपनी जनरेटिव एआई की जानकारी को और बेहतर बनाएं!



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। यद्यपि हम सटीकता के लिए प्रयासरत हैं, कृपया ध्यान दें कि स्वचालित अनुवादों में त्रुटियाँ या गलतियाँ हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को ही प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
